In [56]:
import pandas as pd
dataset = pd.read_csv('mnist_train.csv')

print(len(dataset))

60000


In [69]:
import numpy as np

data = np.array(dataset)
m, n = data.shape

print(data[:3])

print(m, n)

[[5 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [4 0 0 ... 0 0 0]]
60000 785


In [58]:

import random

random.shuffle(data)



[[5 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [4 0 0 ... 0 0 0]]
[[5 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]]


In [77]:
validate = data[:1000]
train = data[1000:]


validate = validate.transpose()
y_validate = validate[0]
x_validate = validate[1:]
x_validate = x_validate / 255

print(y_validate.shape, x_validate.shape)

train = train.transpose()
y_train = train[0]
x_train = train[1:]
x_train = x_train / 255

print(y_train.shape, x_train.shape)



(1000,) (784, 1000)
(59000,) (784, 59000)


In [97]:
#forward propogation: 

#784 -(ReLu)-> 10 - Softmax -> 10 -> Output
def initiate_param():
    w1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    w2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5

    print("This is b1", b1)
    return w1, b1, w2, b2
def Relu(z):
    return np.maximum(0, z)

def softmax(z):
    val = (np.exp(z)) / (np.sum(np.exp(z), axis = 0))
    return val
def forward_prop(W1, b1, W2, b2, X):
    z1 = np.dot(W1, X) + b1
    a1 = Relu(z1)
    z2 = np.dot(W2, a1) + b2
    a2 = softmax(z2)

    return (z1, a1, z2, a2)



#Backwards Prop
def one_hot(Y):
    one_hot_y = np.zeros((10, Y.size))
    one_hot_y[Y, np.arange(Y.size)] = 1
    
    return one_hot_y

def deriv_ReLu(Z):
    return Z > 0

def backprop(Z1, A1, Z2, A2, W2, X, Y):
    m = len(Y)
    one_hot_y = one_hot(Y)
    dZ2 = A2 - one_hot_y
    dW2 = 1 / m * np.dot(dZ2, A1.transpose())
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = np.dot(W2.transpose(), dZ2) * deriv_ReLu(Z1)
    dW1 = 1 / m * np.dot(dZ1, X.transpose())
    db1 = 1 / m * np.sum(dZ1)

    return db1, dW1, db2, dW2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1 
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, iterations, alpha):
    w1, b1, w2, b2 = initiate_param()
    for i in range(iterations):
        z1, a1, z2, a2 = forward_prop(w1, b1, w2, b2, X)
        db1, dw1, db2, dw2 = backprop(z1, a1, z2, a2, w2, X, Y)

        w1, b1, w2, b2 = update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha)
        # print("This is new weights", w1, b1, w2,)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(a2)
            print(get_accuracy(predictions, Y))
    return w1, b1, w2, b2

    
    

In [92]:
gradient_descent(x_train, y_train, 500, 0.1)

This is b1 [[-0.83643469]
 [-0.85068874]
 [-0.4017844 ]
 [-0.91396279]
 [ 0.12491217]
 [ 0.53687317]
 [ 0.58482842]
 [-0.72111551]
 [ 0.12596904]
 [-0.42915364]]
Iteration:  0
[7 7 7 ... 7 7 7] [0 7 1 ... 5 6 8]
0.10420338983050847
Iteration:  10
[7 7 7 ... 7 7 7] [0 7 1 ... 5 6 8]
0.10420338983050847
Iteration:  20
[7 7 7 ... 7 7 7] [0 7 1 ... 5 6 8]
0.10420338983050847
Iteration:  30
[7 7 7 ... 7 7 7] [0 7 1 ... 5 6 8]
0.10420338983050847
Iteration:  40
[7 7 7 ... 7 7 7] [0 7 1 ... 5 6 8]
0.10420338983050847
Iteration:  50
[7 7 7 ... 7 7 7] [0 7 1 ... 5 6 8]
0.10420338983050847


KeyboardInterrupt: 

In [98]:
gradient_descent(x_train, y_train, 500, 0.1)

This is b1 [[-0.376245  ]
 [ 0.02590288]
 [ 0.23547003]
 [-0.00768281]
 [ 0.06496775]
 [-0.03411142]
 [-0.3486838 ]
 [ 0.17895231]
 [-0.13372522]
 [ 0.06393105]]
Iteration:  0
0.07284745762711864
Iteration:  10
0.24964406779661016
Iteration:  20
0.328271186440678
Iteration:  30
0.39888135593220336
Iteration:  40
0.47203389830508474
Iteration:  50
0.5311694915254237
Iteration:  60
0.5770338983050848
Iteration:  70
0.6115593220338983
Iteration:  80
0.636864406779661
Iteration:  90
0.6581864406779661
Iteration:  100
0.6746610169491526


KeyboardInterrupt: 

(59000,)
59000


(10, 59000)
(10, 59000)
